<a href="https://colab.research.google.com/github/LAMECH9/LAMECH9/blob/main/CNN_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil
from tqdm import tqdm
from google.colab import files
import zipfile

In [ ]:

# Function to upload and extract the dataset
def prepare_dataset(zip_filename='chicken_diseases.zip', extract_to='chicken_diseases_dataset'):
    # Upload the zip file (for Google Colab)
    # If running locally, you can skip this and place the zip file manually
    uploaded = files.upload()

    # Check if the zip file exists
    if zip_filename not in uploaded and not os.path.exists(zip_filename):
        raise FileNotFoundError(f"Zip file '{zip_filename}' not found!")

    # Create directory to extract to
    os.makedirs(extract_to, exist_ok=True)

    # Extract the zip file
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

    print(f"Dataset extracted to: {extract_to}")

    # Optional: Remove the zip file after extraction
    os.remove(zip_filename)

    return extract_to

# Prepare the dataset
dataset_path = prepare_dataset()

# Verify the extracted files
print("\nExtracted files:")
for root, dirs, files in os.walk(dataset_path):
    level = root.replace(dataset_path, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 4 * (level + 1)
    for f in files[:5]:  # Print first 5 files in each directory
        print(f'{subindent}{f}')
    if len(files) > 5:
        print(f'{subindent}... and {len(files)-5} more files')

Saving chicken_diseases.zip to chicken_diseases.zip
Dataset extracted to: chicken_diseases_dataset

Extracted files:
chicken_diseases_dataset/
    train_data.csv
    Train/
        salmo.1358.jpg
        cocci.2029.jpg
        salmo.686.jpg
        healthy.860.jpg
        healthy.1722.jpg
        ... and 8062 more files


### 1. Organize the data into training/validation/test **sets**

# Split the Dataset Using CSV

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil
from tqdm import tqdm

# Load labels
df = pd.read_csv("chicken_diseases_dataset/train_data.csv")

# Preview
print(df.head())

# Split
train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)


           images               label
0  salmo.1558.jpg          Salmonella
1  cocci.1866.jpg         Coccidiosis
2   cocci.171.jpg         Coccidiosis
3  salmo.1484.jpg          Salmonella
4     ncd.100.jpg  New Castle Disease


#Create Folder Structure

In [ ]:
def make_dirs(base_dir, labels):
    for label in labels:
        os.makedirs(os.path.join(base_dir, label), exist_ok=True)

make_dirs("organized_data/train", df['label'].unique())
make_dirs("organized_data/val", df['label'].unique())
make_dirs("organized_data/test", df['label'].unique())


#  Copy Files to New Folders

In [ ]:
def move_images(df, subset):
    # Assuming the column containing filenames is called 'images'
    for _, row in tqdm(df.iterrows(), total=len(df)):
        src = os.path.join("chicken_diseases_dataset/Train", row['images'])  # Changed 'filename' to 'images'
        dst = os.path.join("organized_data", subset, row['label'], row['images'])  # Changed 'filename' to 'images'
        if os.path.exists(src):
            shutil.copy(src, dst)

move_images(train_df, "train")
move_images(val_df, "val")
move_images(test_df, "test")

100%|██████████| 807/807 [00:00<00:00, 4145.52it/s]


# Load into CNN Using Keras

In [ ]:
# 3. Data Generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    "organized_data/train",
    target_size=(128, 128),  # Increased size
    batch_size=32,
    class_mode='categorical'
)
val_generator = val_test_datagen.flow_from_directory(
    "organized_data/val",
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)
test_generator = val_test_datagen.flow_from_directory(
    "organized_data/test",
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 6453 images belonging to 4 classes.
Found 807 images belonging to 4 classes.
Found 807 images belonging to 4 classes.


In [ ]:
# 4. Build CNN Model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),  # Added layer
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),  # Increased units
    layers.Dropout(0.4),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# 5. Compile Model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# 6. Train Model
callbacks = [
    callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    callbacks.ModelCheckpoint('best_model.h5', save_best_only=True)
]
history = model.fit(
    train_generator,
    epochs=50,  # Increased with early stopping
    validation_data=val_generator,
    callbacks=callbacks
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 0.5431 - loss: 1.1203

202/202 ━━━━━━━━━━━━━━━━━━━━ 41s 183ms/step - accuracy: 0.5437 - loss: 1.1191 - val_accuracy: 0.7757 - val_loss: 0.6444
Epoch 2/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step - accuracy: 0.7475 - loss: 0.7278

202/202 ━━━━━━━━━━━━━━━━━━━━ 35s 173ms/step - accuracy: 0.7476 - loss: 0.7277 - val_accuracy: 0.7893 - val_loss: 0.5984
Epoch 3/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 33s 164ms/step - accuracy: 0.7863 - loss: 0.6163 - val_accuracy: 0.7695 - val_loss: 0.6424
Epoch 4/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.8011 - loss: 0.5629

202/202 ━━━━━━━━━━━━━━━━━━━━ 32s 158ms/step - accuracy: 0.8012 - loss: 0.5629 - val_accuracy: 0.8290 - val_loss: 0.5115
Epoch 5/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - accuracy: 0.8166 - loss: 0.5314

...

202/202 ━━━━━━━━━━━━━━━━━━━━ 34s 168ms/step - accuracy: 0.8809 - loss: 0.3298 - val_accuracy: 0.9033 - val_loss: 0.2984
Epoch 14/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 32s 159ms/step - accuracy: 0.8862 - loss: 0.2999 - val_accuracy: 0.8922 - val_loss: 0.3205
Epoch 15/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 31s 155ms/step - accuracy: 0.8921 - loss: 0.3050 - val_accuracy: 0.8798 - val_loss: 0.3439
Epoch 16/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - accuracy: 0.9014 - loss: 0.2862

202/202 ━━━━━━━━━━━━━━━━━━━━ 33s 163ms/step - accuracy: 0.9014 - loss: 0.2862 - val_accuracy: 0.8971 - val_loss: 0.2971
Epoch 17/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - accuracy: 0.9044 - loss: 0.2716

202/202 ━━━━━━━━━━━━━━━━━━━━ 33s 162ms/step - accuracy: 0.9044 - loss: 0.2716 - val_accuracy: 0.9182 - val_loss: 0.2627
Epoch 18/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.9065 - loss: 0.2676

202/202 ━━━━━━━━━━━━━━━━━━━━ 32s 160ms/step - accuracy: 0.9065 - loss: 0.2675 - val_accuracy: 0.9170 - val_loss: 0.2444
Epoch 19/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 32s 160ms/step - accuracy: 0.9022 - loss: 0.2752 - val_accuracy: 0.9145 - val_loss: 0.2487
Epoch 20/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 32s 158ms/step - accuracy: 0.9072 - loss: 0.2622 - val_accuracy: 0.9021 - val_loss: 0.2901
Epoch 21/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - accuracy: 0.9116 - loss: 0.2489

202/202 ━━━━━━━━━━━━━━━━━━━━ 33s 161ms/step - accuracy: 0.9117 - loss: 0.2489 - val_accuracy: 0.9343 - val_loss: 0.1986
Epoch 22/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 32s 158ms/step - accuracy: 0.9140 - loss: 0.2461 - val_accuracy: 0.8996 - val_loss: 0.3267
Epoch 23/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 31s 153ms/step - accuracy: 0.9211 - loss: 0.2261 - val_accuracy: 0.9306 - val_loss: 0.2310
Epoch 24/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 32s 160ms/step - accuracy: 0.9209 - loss: 0.2350 - val_accuracy: 0.9170 - val_loss: 0.2935
Epoch 25/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 32s 159ms/step - accuracy: 0.9220 - loss: 0.2310 - val_accuracy: 0.8748 - val_loss: 0.4143
Epoch 26/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 31s 156ms/step - accuracy: 0.9143 - loss: 0.2428 - val_accuracy: 0.9331 - val_loss: 0.2471


In [ ]:
# 7. Evaluate Model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.2f}")

26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.9487 - loss: 0.1653
Test Accuracy: 0.92


In [ ]:
ls -l *.h5

-rw-r--r-- 1 root root 78252288 May  8 18:24 best_model.h5


In [ ]:
!ls -lh /content/chicken_diseases_dataset/*.csv

-rw-r--r-- 1 root root 215K May  8 16:38 /content/chicken_diseases_dataset/train_data.csv


In [ ]:
from google.colab import files
files.download('/content/best_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download('/content/chicken_diseases_dataset/train_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!git lfs install
!git lfs track "*.h5"
!git add .gitattributes best_model.h5

Git LFS initialized.
Not in a git repository.
fatal: not a git repository (or any of the parent directories): .git


In [ ]:
import gdown
if not os.path.exists('best_model.h5'):
    gdown.download('https://drive.google.com/file/d/15nBdRGqmT06D8T9dc9egjVClyGR1In-9/view?usp=drive_link', 'best_model.h5', quiet=False)

In [ ]:
https://drive.google.com/uc?id=15nBdRGqmT06D8T9dc9egjVClyGR1In-9